# 1. Set Up the PyTorch Model

* Here's an example with a basic CNN for image classification (like on the MNIST dataset):

In [18]:
import torch
import torchvision
import copy  # Import the copy module for deep copying objects
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split  # Make sure this import is correct
from torchvision import datasets  # This imports the MNIST dataset
import numpy as np


# Define Simple NN

In [19]:
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.inputlayer = nn.Linear(28*28, 128)
        self.outputlayer = nn.Linear(128, 10)
        
    def forward(self, xb):
        xb = xb.view(-1, 28*28)# flatten the input
        out = F.relu(self.inputlayer(xb))
        out = self.outputlayer(out)
        return out

# Loading the data set and simulate the clients

* We'll use the MNIST dataset and divide it among multiple clients to simulate a federated learning environment.

In [20]:
def load_datasets():
    transform = transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize((0.5,),(0.5,))
    ])
    
    train_dataset = datasets.MNIST(root = 'data/', train = True, transform = transform, download = True)
    test_dataset = datasets.MNIST(root = 'data/', train = False, transform = transform, download = True)
    
    return train_dataset, test_dataset

def split_data(train_dataset, num_clients):
    length = len(train_dataset) // num_clients
    client_datasets = random_split(train_dataset, [length]*num_clients)
    return client_datasets


# Create local training function for each client

* Each client will train their model locally and send the model parameters back to the server

In [21]:
def client_update(client_model, optimizer, train_loader, epochs=1):
    """Train the client model on the local dataset and return loss and accuracy."""
    client_model.train()
    criterion = nn.CrossEntropyLoss()
    total_loss = 0
    correct = 0
    total = 0
    
    for epoch in range(epochs):
        for images, labels in train_loader:
            optimizer.zero_grad()
            output = client_model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(train_loader.dataset)
    accuracy = 100 * correct / total
    
    return client_model.state_dict(), avg_loss, accuracy


# Define the server aggregation function

* The server will aggregate the weights of all clients using weighted averaging. This step assumes that all clients have equal amounts of data; otherwise, the aggregation would need to be adjusted.

In [22]:
def server_aggregate(global_model, client_weights):
    """Aggregate the client models into the global model."""
    global_dict = global_model.state_dict()
    
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_weights[i][k].float() for i in range(len(client_weights))], 0).mean(0)
    
    global_model.load_state_dict(global_dict)
    return global_model


# main Federated Learning Process

* In this step, we initialize the global model, split the dataset among clients, and perform multiple rounds of federated learning.

In [23]:
def federated_learning(train_dataset, num_clients=5, num_rounds=5, epochs=1):
    # Initialize global model
    global_model = SimpleNN()
    
    # Split the dataset between clients
    client_datasets = split_data(train_dataset, num_clients)
    
    # Create data loaders for each client
    client_loaders = [DataLoader(client_datasets[i], batch_size=32, shuffle=True) for i in range(num_clients)]
    
    for round in range(num_rounds):
        print(f"Round {round+1}/{num_rounds}")
        
        client_weights = []
        client_losses = []
        client_accuracies = []
        
        # Train each client model
        for client_idx in range(num_clients):
            # DeepCopy: Creates a completely independent duplicate of the object. 
            #Any changes to the copy do not affect the original object
            client_model = copy.deepcopy(global_model)
            optimizer = optim.SGD(client_model.parameters(), lr=0.01)
            
            # Update client model and get loss and accuracy
            client_weight, client_loss, client_accuracy = client_update(client_model, optimizer, client_loaders[client_idx], epochs)
            client_weights.append(client_weight)
            client_losses.append(client_loss)
            client_accuracies.append(client_accuracy)
        
        # Aggregate client models
        global_model = server_aggregate(global_model, client_weights)
        
        # Calculate and print the average loss and accuracy for the round
        avg_loss = np.mean(client_losses)
        avg_accuracy = np.mean(client_accuracies)
        
        print(f"Average Loss: {avg_loss:.4f}, Average Accuracy: {avg_accuracy:.2f}%")
    
    return global_model


# Test the global model

* Once federated learning rounds are complete, you can evaluate the global model on a central test dataset.

In [24]:
def test(global_model, test_loader):
    global_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = global_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')


# Running the federated learning process

In [25]:
if __name__ == "__main__":
    # Load datasets
    train_dataset, test_dataset = load_datasets()
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Perform federated learning
    global_model = federated_learning(train_dataset, num_clients=5, num_rounds=5, epochs=1)
    
    # Test the global model
    test(global_model, test_loader)


Round 1/5
Average Loss: 0.0360, Average Accuracy: 72.80%
Round 2/5
Average Loss: 0.0160, Average Accuracy: 86.46%
Round 3/5
Average Loss: 0.0130, Average Accuracy: 88.43%
Round 4/5
Average Loss: 0.0117, Average Accuracy: 89.31%
Round 5/5
Average Loss: 0.0110, Average Accuracy: 89.88%
Test Accuracy: 90.98%
